# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [ ]:
!apt-get update && apt-get install -y python3-opencv

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
!pip install sklearn

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Feb 10 08:22:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [38]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from pylab import rcParams
from google.colab import drive
drive.mount('/content/drive/')
from google.colab.patches import cv2_imshow 
import torchvision.transforms.functional as TF
from imgaug import augmentables
import imgaug as ia
import imgaug.augmenters as iaa


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Set Arguments & hyperparameters

In [39]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [52]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= '/content/drive/MyDrive/이어드림/project2/data'
NUM_CLS = 2
EPOCHS = 30
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 128
os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda')
DATA_SIZE = 2000
TRAIN_PER = 0.7

## Dataloader

#### Train & Validation Set loader

In [ ]:
# !unzip -uq '/content/drive/MyDrive/이어드림/project2/test.zip' -d '/content/drive/MyDrive/이어드림/project2'
# !unzip -uq '/content/drive/MyDrive/이어드림/project2/train.zip' -d '/content/drive/MyDrive/이어드림/project2'


In [49]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape, data_size, train_per):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        self.data_size = data_size
        self.train_per = train_per
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int((self.data_size) * (self.train_per))]
        elif self.mode == 'val':
            self.db = self.db[int((self.data_size) * (self.train_per)):int(self.data_size)]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        def dbplus(db):
            augdb = db.copy()
            for i in range(15):
                if i == 0:
                    augdb['file_name'] = db['file_name'].apply(lambda x : int(x[:-4]) + int('646'))
                    augdb['file_name'] = augdb['file_name'].apply(lambda x : str(x) +'.png')
                    db = pd.concat([db, augdb], ignore_index = True,axis=0)
                else :
                    augdb['file_name'] = augdb['file_name'].apply(lambda x : int(x[:-4]) + int('646'))
                    augdb['file_name'] = augdb['file_name'].apply(lambda x : str(x) +'.png')
                    db = pd.concat([db, augdb], ignore_index = True,axis=0)
            return db
        return dbplus(db)
        
    def __len__(self):
        if self.mode == 'train':
            return int(int(self.data_size)*(self.train_per))
        elif self.mode == 'val':
            return int(int(self.data_size) - (int(self.data_size)*(self.train_per)))
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name+'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))
         
        return trans_image, data['COVID']

    

In [ ]:
# #########Equalization visualization###################
# import imgaug as ia

# db = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
# data = copy.deepcopy(db.loc[7])
# sss =cv2.imread(os.path.join(DATA_DIR,'train',data['file_name']), 0)
# cv2_imshow(sss)
# print(sss.shape)
# augmentation_resize = iaa.Sequential([
#                 iaa.Resize({"height":384,"width":384},interpolation="cubic")
#             ])


# image_aug = augmentation_resize(image=sss)
# cv2_imshow(image_aug)
# # sometimes = lambda aug: iaa.Sometimes(0.5, aug)
# augmentation_resize = iaa.Sequential([
#                     # iaa.Affine(translate_percent={"x":(-0.5,0.5),"y":(-0.5,0.5)},rotate=(-2,2),scale=(0.5,2)),
#                     iaa.Crop(px=(0, 16)),
#                     iaa.Multiply((0.8, 1.3)),
#                     iaa.LinearContrast((1.5,2.3)),
#                     # iaa.GaussianBlur((0.0,3.0)),
#                     iaa.Fliplr(0.5),
#                     iaa.Flipud(0.2),
#                     iaa.Sharpen(alpha=(0, 1.0), lightness=(0.95, 1.5)),
#                     iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0))])
      
                    
# image_augs = augmentation_resize(image=image_aug)
# cv2_imshow(image_augs)

                    

In [46]:
#augment시 사용
eq_img_list = []
class Augmentation:
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        self.img = None
    
        #이미지크기 저장공간

        self.db = self.data_loader()
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
       
        return db
    def Equalization(self):
    #    // for cnt in tqdm(range(646)):
        for i in tqdm(range(646)):
            # img = eq_img_list
            db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
            data = copy.deepcopy(db.loc[i])
            cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']))
            cvimg_yuv = cv2.cvtColor(cvimg, cv2.COLOR_BGR2YUV)

            img_clahe = cvimg_yuv.copy()
            clahe = cv2.createCLAHE(clipLimit = 3.0 , tileGridSize=(8,8))
            img_clahe[:,:,0] = clahe.apply(img_clahe[:,:,0])
            img_clahe = cv2.cvtColor(img_clahe, cv2.COLOR_YUV2BGR)

                # img_eq = cv2_imshow(img_clahe)
            # cv2.waitKey()
            cv2.destroyAllWindows()
            eq_img_list.append(img_clahe)
            # os.makedirs('/content/drive/MyDrive/이어드림/project2/data/aug_img', exist_ok=True)
            cv2.imwrite(f'/content/drive/MyDrive/이어드림/project2/data/train/{i + 646}.png', img_clahe)
            self.img = eq_img_list
        return eq_img_list

    def augmentation(self):
        
        for i in tqdm(range(646)):
            
            db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
            data = copy.deepcopy(db.loc[i])
            sss =cv2.imread(os.path.join(self.data_dir,'train',data['file_name']))
            
            # cv2_imshow(sss)
            augmentation_resize = iaa.Sequential([
                            iaa.Resize({"height":384,"width":384},interpolation="cubic")
                        ])


            image_aug = augmentation_resize(image=sss)
            # cv2_imshow(image_aug)
            # sometimes = lambda aug: iaa.Sometimes(0.5, aug)
            for j in (range(30)):
                augmentation_resize = iaa.Sequential([
                                    # iaa.Affine(translate_percent={"x":(-0.5,0.5),"y":(-0.5,0.5)},rotate=(-2,2),scale=(0.5,2)),
                                    iaa.Crop(px=(0, 16)),
                                    iaa.Multiply((0.8, 1.3)),
                                    iaa.LinearContrast((1.5,2.3)),
                                    # iaa.GaussianBlur((0.0,3.0)),
                                    iaa.Fliplr(0.5),
                                    iaa.Flipud(0.5),
                                    iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)}),
                                    iaa.PerspectiveTransform(scale=(0.01, 0.15)),
                                    iaa.PiecewiseAffine(scale=(0.01, 0.05)),
                                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.95, 1.5)),
                                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0))])
                    
                image_augs = augmentation_resize(image=image_aug)
            # cv2_imshow(image_augs)
                # os.makedirs('/content/drive/MyDrive/이어드림/project2/data/augment{}'.format(j), exist_ok=True)
                cv2.imwrite(f'/content/drive/MyDrive/이어드림/project2/data/train/{(i)+(1292)+(646*j)}.png', image_augs)


In [47]:
#data augm
if __name__ == '__main__': 
    temp = Augmentation(DATA_DIR,'train', INPUT_SHAPE)
    print('이미w지를 불러오는 중입니다.')
    temp.data_loader()
    print('*' * 30)
    print('Equalization중 입니다')
    #temp.Equalization()
    print('*' * 30)
    print('Equalization 성공')
    print('Augmentation중 입니다')
    temp.augmentation()

    print('*' * 30)
    del temp
    

Loading train dataset..
이미w지를 불러오는 중입니다.
Loading train dataset..
******************************
Equalization중 입니다
******************************
Equalization 성공
Augmentation중 입니다


 52%|█████▏    | 337/646 [46:10<42:20,  8.22s/it]


KeyboardInterrupt: ignored

## Model

In [19]:
import torch.nn.functional as F

class custom_CNN(nn.Module):
    def __init__(self, num_classes):
        super(custom_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=25*29*29, out_features=128)
        #self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features=128, out_features=num_classes)

        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
        x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        #x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        
        output = self.softmax(x)
        
        return output

## Utils
### EarlyStopper

In [20]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [21]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [22]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [53]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE, data_size= DATA_SIZE, train_per = TRAIN_PER)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE, data_size = DATA_SIZE, train_per = TRAIN_PER)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 1400 Val set samples: 600


In [ ]:
!pip install timm

In [ ]:
import timm 
#기본 사용시 0.766
#efficientnet
class Efficientnet(nn.Module):
    def __init__(self, num_classes):
        super(Efficientnet, self).__init__()
        self.efficientnet = timm.create_model('efficientnetv2_s', pretrained=False, num_classes=num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.efficientnet(x)
        
        output = self.softmax(x)
        
        return output

#### Load model and other utils

In [30]:
# Load Model
model = custom_CNN(NUM_CLS).to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [31]:
model

custom_CNN(
  (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 25, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=21025, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

### epoch 단위 학습 진행

In [32]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.6769338416498761, Acc: 0.5992857142857143, F1-Macro: 0.5896069287141905


  3%|▎         | 1/30 [00:17<08:26, 17.47s/it]

Epoch 0, Val loss: 0.6651052369011773, Acc: 0.655, F1-Macro: 0.6542999574137629
Epoch 1, Train loss: 0.4904471781364707, Acc: 0.7692857142857142, F1-Macro: 0.768471940406869


  7%|▋         | 2/30 [00:34<08:07, 17.42s/it]

...

Epoch 8, Val loss: 1.952772715025478, Acc: 0.7416666666666667, F1-Macro: 0.7412865015536719
Early stopping counter 8/10
Epoch 9, Train loss: 0.0005451430008898294, Acc: 1.0, F1-Macro: 1.0


 33%|███▎      | 10/30 [02:54<05:48, 17.40s/it]

Epoch 9, Val loss: 2.16565563943651, Acc: 0.7383333333333333, F1-Macro: 0.7381696893892016
Early stopping counter 9/10
Epoch 10, Train loss: 0.00044038678065272657, Acc: 1.0, F1-Macro: 1.0


 33%|███▎      | 10/30 [03:11<06:23, 19.17s/it]

Epoch 10, Val loss: 2.267538345522351, Acc: 0.7433333333333333, F1-Macro: 0.743047830923248
Early stopping counter 10/10
Early stopped


## Inference
### 모델 로드

In [33]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [34]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [35]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [36]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:00,  3.43it/s]

tensor([[3.3233e-01, 6.6767e-01],
        [9.8640e-01, 1.3602e-02],
        [9.9258e-01, 7.4226e-03],
        [1.8353e-04, 9.9982e-01],
        [7.7023e-01, 2.2977e-01],
        [2.8520e-03, 9.9715e-01],
        [2.1277e-03, 9.9787e-01],
        [7.7942e-02, 9.2206e-01],
        [1.0000e+00, 3.7712e-07],
        [5.3618e-07, 1.0000e+00],
        [6.0757e-01, 3.9243e-01],
        [8.5285e-01, 1.4715e-01],
        [9.7628e-02, 9.0237e-01],
        [1.0000e+00, 5.1732e-08],
        [9.9976e-01, 2.4376e-04],
        [9.9583e-01, 4.1667e-03],
        [7.2966e-01, 2.7034e-01],
        [1.0000e+00, 4.0393e-06],
        [9.9765e-01, 2.3540e-03],
        [9.9606e-01, 3.9383e-03],
        [1.0000e+00, 2.9629e-07],
        [9.9657e-01, 3.4350e-03],
        [9.7694e-01, 2.3063e-02],
        [1.8252e-01, 8.1748e-01],
        [1.0000e+00, 1.5197e-06],
        [1.0000e+00, 6.7776e-07],
        [1.0000e+00, 1.6590e-07],
        [9.9968e-01, 3.2456e-04],
        [1.0541e-05, 9.9999e-01],
        [2.850

2it [00:00,  3.49it/s]

tensor([[8.9685e-01, 1.0315e-01],
        [3.1713e-01, 6.8287e-01],
        [9.9967e-01, 3.2719e-04],
        [1.1149e-01, 8.8851e-01],
        [9.9997e-01, 3.1127e-05],
        [6.9143e-03, 9.9309e-01],
        [1.0195e-01, 8.9805e-01],
        [2.3974e-01, 7.6026e-01],
        [1.5435e-05, 9.9998e-01],
        [4.2047e-02, 9.5795e-01],
        [6.6023e-01, 3.3977e-01],
        [6.5953e-03, 9.9340e-01],
        [9.6547e-01, 3.4534e-02],
        [5.8866e-06, 9.9999e-01],
        [9.4982e-01, 5.0181e-02],
        [9.9903e-01, 9.7241e-04],
        [3.0732e-01, 6.9268e-01],
        [9.4565e-01, 5.4353e-02],
        [4.5446e-02, 9.5455e-01],
        [9.4744e-03, 9.9053e-01],
        [8.8497e-01, 1.1503e-01],
        [8.7558e-01, 1.2442e-01],
        [5.0508e-05, 9.9995e-01],
        [9.9999e-01, 1.0541e-05],
        [9.1329e-01, 8.6715e-02],
        [5.4124e-04, 9.9946e-01],
        [6.6639e-01, 3.3361e-01],
        [2.7223e-01, 7.2777e-01],
        [5.2676e-01, 4.7324e-01],
        [9.999

4it [00:00,  4.45it/s]

tensor([[1.1087e-05, 9.9999e-01],
        [4.7387e-07, 1.0000e+00],
        [7.2529e-05, 9.9993e-01],
        [9.9995e-01, 4.6430e-05],
        [1.5408e-05, 9.9998e-01],
        [9.6515e-01, 3.4854e-02],
        [9.9909e-01, 9.0870e-04],
        [9.9963e-01, 3.7265e-04],
        [9.9999e-01, 6.9930e-06],
        [9.9973e-01, 2.6667e-04],
        [1.6080e-10, 1.0000e+00],
        [9.9923e-01, 7.7173e-04],
        [1.0000e+00, 5.5726e-07],
        [8.2528e-01, 1.7472e-01],
        [4.7087e-01, 5.2913e-01],
        [1.3722e-01, 8.6278e-01],
        [1.0981e-07, 1.0000e+00],
        [5.3858e-01, 4.6142e-01],
        [5.8648e-01, 4.1352e-01],
        [2.6292e-01, 7.3708e-01],
        [1.0552e-01, 8.9448e-01],
        [9.9996e-01, 3.6144e-05],
        [9.9969e-01, 3.1389e-04],
        [9.9978e-01, 2.2202e-04],
        [6.1438e-01, 3.8562e-01],
        [9.9913e-01, 8.6834e-04],
        [9.9999e-01, 8.5097e-06],
        [9.3646e-01, 6.3544e-02],
        [9.9921e-01, 7.8874e-04],
        [1.377

### 결과 저장

In [37]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv(DATA_DIR+'/prediction(size3000,7:3).csv', index=False)
#과적합줄이기